In [1]:
import csv
import pandas as pd

In [2]:
from estnltk import Layer
from estnltk.taggers import Tagger
from collections import defaultdict
import copy

In [3]:
def get_ner(ner_layer, word_layer, span):
        nertag = None
        if len(ner_layer) > 0:
            word = word_layer.get(span)
            for n in ner_layer:
                for part in n:
                    if part==word:
                        nertag=n.nertag
        if nertag:
            return nertag
        return 'OTHER'
    
def get_POS(word_layer, span):
    infinite_verb_forms = ['da', 'des', 'ma', 'maks', 'mas', 'mast', 'mata', 'nud', 'tav', 'tud', 'v']
    # if POS is ambiguous, only unique tags are kept, e.g. ['V', 'A', 'A'] -> ['V', 'A']
    pos_list = []
    word = word_layer.get(span)
    for i in range(len(word.morph_analysis['partofspeech'])):
        if word.morph_analysis['partofspeech'][i] == 'V':
            if word.morph_analysis['form'][i] in infinite_verb_forms:
                pos_list.append('V_inf')
            elif word.form[i] == 'neg':
                pos_list.append('V_neg')
            else:
                pos_list.append('V_fin')
        else:
            pos_list.append(word.morph_analysis['partofspeech'][i])
    
    if len(pos_list) > 1:
        char_unique = [char for indx, char in enumerate(pos_list) if char not in pos_list[:indx]]
        if len(char_unique) < 2:
            return char_unique[0]
        return '|'.join(char_unique)
    return pos_list[0]

In [13]:
class PhrasePatternConsistencyTagger(Tagger):
    """Tags phrase words that are syntactically wrong or have wrong part-of-speech.""" 
    conf_param = ['rules_file', 'ruleset_map']
    
    def __init__(self, rules_file: str,
                       output_layer='pattern_consistency',
                       morph_analysis_layer='morph_analysis',
                       words_layer='words',
                       syntax_layer='stanza_syntax',
                       ner_layer='ner'):
        
        self.input_layers = [morph_analysis_layer, words_layer, syntax_layer, ner_layer]
        self.output_layer = output_layer
        self.output_attributes = ['phrase', 'syntax', 'pos', 'ner', 'error_source', 'error_mask', 'correction']
        self.rules_file = rules_file
        
        ruleset_map = defaultdict(list)
        
        with open(rules_file, encoding='UTF-8') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                info = [row['POS_pattern'], row['NER_pattern'], row['mistake_type'], row['error_mask'], row['correction']]
                ruleset_map[row['tree']].append(info)
                
        self.ruleset_map = ruleset_map

    def _make_layer_template(self):
        # Create new detached layer debased on the configuration
        layer = Layer(name=self.output_layer,
                      text_object=None,
                      attributes=self.output_attributes,
                      enveloping=self.input_layers[1],
                      ambiguous=True)
        return layer
    
    # to be continued...
        
    def _make_layer(self, text, layers, status):
        # Create new layer based on the configuration
        layer = self._make_layer_template()
        # Assign the Text object
        layer.text_object = text
        
        for i in range(len(layers[self.input_layers[2]])): # Iterate over 'stanza_syntax' layer
            pattern_spans = []
            tree = []
            ids = []
            pos = []
            ner = []

            pattern_spans.append(layers[self.input_layers[2]][i])
            ids.append([layers[self.input_layers[2]][i]['id'], layers[self.input_layers[2]][i]['head']])
                
            for j in range(i + 1, len(layers[self.input_layers[2]])):
                for k in range(len(pattern_spans)):
                    if layers[self.input_layers[2]][j] in pattern_spans[k]['children'] or pattern_spans[k] in layers[self.input_layers[2]][j]['children'] or layers[self.input_layers[2]][j]['parent_span'] != None and layers[self.input_layers[2]][j]['parent_span'] == pattern_spans[k]['parent_span']:
                        pattern_spans.append(layers[self.input_layers[2]][j])
                        ids.append([layers[self.input_layers[2]][j]['id'], layers[self.input_layers[2]][j]['head']])
        
                ids_for_pattern = copy.deepcopy(ids)
                for k in range(len(ids_for_pattern)):
                    temp = ids_for_pattern[k][0]
                    ids_for_pattern[k][0] = k+1
                    for l in range(len(ids)):
                        if ids[l][1] == temp:
                            ids_for_pattern[l][1] = ids_for_pattern[k][0]
            
                word_ids = [word_id[0] for word_id in ids_for_pattern]
                for k in range(len(ids_for_pattern)):
                    if ids_for_pattern[k][0] == ids_for_pattern[k][1]:
                        ids_for_pattern[k][1] = 0
                    elif ids_for_pattern[k][1] not in word_ids:
                        ids_for_pattern[k][1] = 0
            
                for k in range(len(pattern_spans)):
                    deprel = pattern_spans[k].deprel
                    if ids_for_pattern[k][1] == 0 and deprel != 'root':
                        deprel = 'root'
                    tree.append([str(ids_for_pattern[k][0]), str(ids_for_pattern[k][1]), deprel])
                    # POS-tag is taken from morph_analysis layer
                    pos.append(get_POS(layers[self.input_layers[1]], pattern_spans[k]))
                    # nertag is taken from ner layer
                    ner.append(get_ner(layers[self.input_layers[-1]], layers[self.input_layers[1]], pattern_spans[k]))                     
                    
                pattern = [" ".join(word_info) for word_info in tree]
                # check if tree pattern exists in ruleset map
                if ",".join(pattern) in self.ruleset_map.keys():
                    pos_pattern = "-".join(pos)
                    ner_pattern = "-".join(ner)
                    # check if POS-sequence exists in ruleset map with given tree pattern
                    for el in self.ruleset_map[",".join(pattern)]:
                        if el[0] == pos_pattern and el[1] == ner_pattern:
                            # add annotations
                            for idx, span in enumerate(pattern_spans):
                                layer.add_annotation([span.base_span], 
                                                 phrase=" ".join([span.text for span in pattern_spans]),
                                                 syntax=pattern[idx],
                                                 pos=pos[idx],
                                                 ner=ner[idx],
                                                 error_source=el[2], 
                                                 error_mask=el[3].split('-')[idx],
                                                 correction=el[4]) # correction is currently "-"                  

                if j == len(layers[self.input_layers[2]]) - 1:
                    pattern_spans = []
                    tree = []
                    ids = []
                    pos = []
                    ner = []
                    break 
        
        return layer

In [14]:
consistency_tagger = PhrasePatternConsistencyTagger(rules_file='indicator_patterns_ner_tree_pos_updated.csv')
consistency_tagger

PhrasePatternConsistencyTagger(input_layers=('morph_analysis', 'words', 'stanza_syntax', 'ner'), output_layer=pattern_consistency, output_attributes=('phrase', 'syntax', 'pos', 'ner', 'error_source', 'error_mask', 'correction'), rules_file=indicator_patterns_ner_tree_pos_updated.csv, ruleset_map=defaultdict(<class 'list'>, {'string': [['string', 'string', 'string', 'string', ..., type: <class 'collections.defaultdict'>, length: 7)

In [15]:
test_df = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases.pkl')

In [16]:
test_500 = []

for idx, row in test_df.iterrows():
    if row['phrase_length'] > 1:
        test_500.append(row['phrase'])
    if len(test_500) == 500:
        break

In [17]:
for phrase in test_500:
    consistency_tagger.tag(phrase)

In [19]:
n_found = 0
for phrase in test_500:
    try:
        if len(phrase.pattern_consistency) > 0:
            for word in phrase.pattern_consistency:
                if word.error_mask[0] == '1':
                    display(phrase)
                    display(phrase.pattern_consistency)
                    n_found+=1
    except:
        break
print(n_found)

Text(text='Defender-18 : Sokrates korras sellega ?')

Layer(name='pattern_consistency', attributes=('phrase', 'syntax', 'pos', 'ner', 'error_source', 'error_mask', 'correction'), spans=SL[EnvelopingSpan(['Sokrates'], [{'phrase': 'Sokrates korras', 'syntax': '1 2 nmod', 'pos': 'H', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '1', 'correction': '-'}]),
EnvelopingSpan(['korras'], [{'phrase': 'Sokrates korras', 'syntax': '2 0 root', 'pos': 'S', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '0', 'correction': '-'}])])

Text(text='Tartu ülikooli')

Layer(name='pattern_consistency', attributes=('phrase', 'syntax', 'pos', 'ner', 'error_source', 'error_mask', 'correction'), spans=SL[EnvelopingSpan(['Tartu'], [{'phrase': 'Tartu ülikooli', 'syntax': '1 2 nmod', 'pos': 'H', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '1', 'correction': '-'}]),
EnvelopingSpan(['ülikooli'], [{'phrase': 'Tartu ülikooli', 'syntax': '2 0 root', 'pos': 'S', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '0', 'correction': '-'}])])

Text(text='TÜ Kliinikumi')

...

Text(text='Euroopa Komisjoni')

Layer(name='pattern_consistency', attributes=('phrase', 'syntax', 'pos', 'ner', 'error_source', 'error_mask', 'correction'), spans=SL[EnvelopingSpan(['Euroopa'], [{'phrase': 'Euroopa Komisjoni', 'syntax': '1 2 nmod', 'pos': 'H', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '1', 'correction': '-'}]),
EnvelopingSpan(['Komisjoni'], [{'phrase': 'Euroopa Komisjoni', 'syntax': '2 0 root', 'pos': 'H', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '1', 'correction': '-'}])])

Text(text='Euroopa Komisjoni')

Layer(name='pattern_consistency', attributes=('phrase', 'syntax', 'pos', 'ner', 'error_source', 'error_mask', 'correction'), spans=SL[EnvelopingSpan(['Euroopa'], [{'phrase': 'Euroopa Komisjoni', 'syntax': '1 2 nmod', 'pos': 'H', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '1', 'correction': '-'}]),
EnvelopingSpan(['Komisjoni'], [{'phrase': 'Euroopa Komisjoni', 'syntax': '2 0 root', 'pos': 'H', 'ner': 'OTHER', 'error_source': 'ner', 'error_mask': '1', 'correction': '-'}])])

41
